In [1]:
# Standard library imports
import os
import asyncio
import logging
from pathlib import Path
from typing import Tuple, Any, List, Dict, Optional
from dataclasses import dataclass
from asyncio import sleep, Semaphore
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type # To implement exponential backoff
import httpx
import numpy as np
from dotenv import load_dotenv

# Embeddings and LLM imports
from langchain_cohere import CohereEmbeddings
from langchain_openai import ChatOpenAI

# Vector store and text splitters imports 
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.docstore import InMemoryDocstore
import faiss

# Document Loader Imports
from langchain_community.document_loaders import (
    TextLoader, 
    UnstructuredMarkdownLoader,
    JSONLoader,
    UnstructuredHTMLLoader,
    PyPDFLoader
)
from langchain_core.documents import Document

# Prompt and template imports
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# Langchain  runnables and pipeline imports
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda

# Callbacks and logging imports
from langchain.callbacks.base import AsyncCallbackHandler
from langchain.schema import LLMResult
from langchain_community.vectorstores import FAISS

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configure environment variables
load_dotenv()


True

In [2]:
@dataclass
class SelfRAGResponse:
    """Complete self rag with reflection"""
    answer: str
    retrieved_docs: List[Document]
    reflection_score: float
    needs_retrieval: bool
    citations: List[str]
    retrieval_decision_reasoning: str

class RateLimitCallback(AsyncCallbackHandler):
    """Callback handler to manage API rate limiting with semaphores"""
    
    def __init__(self, semaphore: asyncio.Semaphore):
        self.semaphore = semaphore
        
    async def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> None:
        await self.semaphore.acquire()
        
    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        self.semaphore.release()


In [3]:
class RateLimitedCohereEmbeddings:
    """Wrapper for Cohere embeddings with rate limiting and retry logic"""
    
    def __init__(self, model: str, cohere_api_key: str, max_concurrent: int = 2, delay_between_calls: float = 0.5, batch_size: int = 30):
        self.base_embeddings = CohereEmbeddings(
            model=model,
            cohere_api_key=cohere_api_key
        )
        self.semaphore = Semaphore(max_concurrent)
        self.delay_between_calls = delay_between_calls
        self.batch_size = batch_size
        self.last_call_time = 0
    
    @retry(
        stop=stop_after_attempt(5),
        wait=wait_exponential(multiplier=1, min=1, max=60),
        retry=retry_if_exception_type((httpx.HTTPStatusError, Exception))
    )
    async def _embed_with_retry(self, texts: List[str]) -> List[List[float]]:
        """Embed texts with retry logic"""
        async with self.semaphore:
            # Ensure minimum delay between calls
            current_time = asyncio.get_event_loop().time()
            time_since_last_call = current_time - self.last_call_time
            if time_since_last_call < self.delay_between_calls:
                await sleep(self.delay_between_calls - time_since_last_call)
            
            try:
                logger.debug(f"Embedding batch of {len(texts)} texts")
                result = await self.base_embeddings.aembed_documents(texts)
                self.last_call_time = asyncio.get_event_loop().time()
                return result
                
            except Exception as e:
                if "429" in str(e) or "Too Many Requests" in str(e):
                    logger.warning(f"Rate limit hit, retrying after delay...")
                    await sleep(2)  # Additional delay for rate limits
                    raise
                else:
                    logger.error(f"Embedding error: {e}")
                    raise
    
    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed documents with batching and rate limiting"""
        if not texts:
            return []
        
        # Use configurable batch size
        all_embeddings = []
        total_batches = (len(texts) + self.batch_size - 1) // self.batch_size
        
        logger.info(f"Processing {len(texts)} texts in {total_batches} batches of {self.batch_size}")
        
        for i in range(0, len(texts), self.batch_size):
            batch_num = (i // self.batch_size) + 1
            batch = texts[i:i + self.batch_size]
            
            logger.info(f"Processing batch {batch_num}/{total_batches} ({len(batch)} texts)")
            
            batch_embeddings = await self._embed_with_retry(batch)
            all_embeddings.extend(batch_embeddings)
            
            # Delay between batches (except for the last one)
            if i + self.batch_size < len(texts):
                await sleep(self.delay_between_calls)
        
        return all_embeddings
    
    async def aembed_query(self, text: str) -> List[float]:
        """Embed a single query"""
        async with self.semaphore:
            current_time = asyncio.get_event_loop().time()
            time_since_last_call = current_time - self.last_call_time
            if time_since_last_call < self.delay_between_calls:
                await sleep(self.delay_between_calls - time_since_last_call)
            
            result = await self.base_embeddings.aembed_query(text)
            self.last_call_time = asyncio.get_event_loop().time()
            return result


In [4]:
class DocumentLoader:
    def __init__(self):
        self.loaders = {
            ".txt": TextLoader,
            ".md": UnstructuredMarkdownLoader,
            ".json": self._create_json_loader,
            ".pdf": PyPDFLoader,
            ".html": UnstructuredHTMLLoader,
            ".py": TextLoader,
            ".js": TextLoader,
            ".css": TextLoader
        }

    def _create_json_loader(self, file_path: str):
        """Create JSON loader with custom jq_schema"""
        return JSONLoader(
            file_path=file_path,
            jq_schema='.[]',
            text_content=False
        )

    async def load_documents(self, kb_folder: str) -> List[Document]:
        """Load all documents from the knowledge base folder"""
        documents = []
        kb_path = Path(kb_folder)

        if not kb_path.exists():
            raise FileNotFoundError(f"Knowledge base folder not found: {kb_path}")

        for file_path in kb_path.glob("**/*"):
            if file_path.is_file() and file_path.suffix.lower() in self.loaders:
                try:
                    loader_class = self.loaders[file_path.suffix.lower()]

                    if file_path.suffix.lower() == ".json":
                        loader = loader_class(str(file_path))
                    else:
                        loader = loader_class(str(file_path))

                    docs = loader.load()

                    # Add metadata
                    for doc in docs:
                        doc.metadata.update({
                            'file_path': str(file_path),
                            'file_type': file_path.suffix,
                            'file_name': file_path.name
                        })

                    documents.extend(docs)
                
                except Exception as e:
                    logger.warning(f"There was an error loading the knowledge base: {str(e)}")
                    # Fallback to TextLoader for unknown formats
                    try:
                        loader = TextLoader(str(file_path))
                        docs = loader.load()
                        # Add metadata
                        for doc in docs:
                            doc.metadata.update({
                                'file_path': str(file_path),
                                'file_type': file_path.suffix,
                                'file_name': file_path.name
                            })

                        documents.extend(docs)

                    except Exception as fallback_error:
                        logger.error(f"Failed to load {file_path} with fallback: {fallback_error}")
        
        logger.info(f"Loaded {len(documents)} documents from {kb_folder}")
        return documents

In [5]:

@dataclass
class RAGSystem:
    cohere_api_key: str
    openrouter_api_key: str
    kb_folder: str
    vector_store_path: str = None
    max_concurrent_requests: int = 3  
    max_concurrent_embeddings: int = 5  
    embedding_delay: float = 0.2  
    embedding_batch_size: int = 96  
    chunk_size: int = 2000
    chunk_overlap: int = 200
    auto_save_vector_store: bool = True

    def __post_init__(self):
        # Initialize with rate-limited embeddings
        self.embeddings = RateLimitedCohereEmbeddings(
            model="embed-v4.0",
            cohere_api_key=self.cohere_api_key,
            max_concurrent=self.max_concurrent_embeddings,
            delay_between_calls=self.embedding_delay,
            batch_size=self.embedding_batch_size
        )

        self.llm = ChatOpenAI(
            model="meta-llama/llama-3.3-70b-instruct",
            openai_api_key=self.openrouter_api_key,
            openai_api_base="https://openrouter.ai/api/v1",
            temperature=0.6,
            max_tokens=1500
        )

        # Text Splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len
        )

        self.document_loader = DocumentLoader()

        # Vector store
        self.vector_store: Optional[FAISS] = None
        
        # Set default vector store path if not provided
        if self.vector_store_path is None:
            self.vector_store_path = os.path.join(self.kb_folder, "vector_store")

        # Semaphores for rate limiting
        self.llm_semaphore = Semaphore(self.max_concurrent_requests)

        self.rate_limit_callback = RateLimitCallback(self.llm_semaphore)

        self.is_initialized = False

        # Set up prompts
        self._setup_prompts()

    def _setup_prompts(self):
        """Set up prompts for different stages"""

        # Retrieval decision prompt
        self.retrieval_decision_prompt = PromptTemplate(
            input_variables=["query"],
            template="""
            Analyze the following query to determine if it requires external knowledge retrieval.
            
            Query: "{query}"
            
            Consider:
            1. Does this query ask for specific facts, data, or domain-specific information?
            2. Would the answer benefit from external documents or knowledge base?
            3. Is this asking about general knowledge that can be answered without retrieval?
            4. Does it require recent or specialized information?
            
            Provide your reasoning and then answer with either "RETRIEVE" or "NO_RETRIEVE".
            
            Reasoning: [Explain your decision]
            Decision: [RETRIEVE or NO_RETRIEVE]
            """
        )

        # Answer generation with retrieval prompt
        self.rag_prompt = ChatPromptTemplate.from_template("""
            You are a helpful AI assistant. Use the following context documents to answer the user's question accurately and comprehensively.
            
            Context Documents:
            {context}
            
            Question: {question}
            
            Instructions:
            - Base your answer primarily on the provided context
            - If the context doesn't contain sufficient information, acknowledge this
            - Cite specific documents when referencing information
            - Be accurate, detailed, and helpful
            - If you need to use general knowledge to supplement the context, clearly indicate this
            
            Answer:
        """)

        # Answer generation without retrieval prompt
        self.no_retrieval_prompt = ChatPromptTemplate.from_template("""
            You are a helpful AI assistant. Answer the following question using your general knowledge.
            
            Question: {question}
            
            Provide a comprehensive and accurate answer based on your training knowledge.
            
            Answer:
        """)

        # Reflection prompt
        self.reflection_prompt = PromptTemplate(
            input_variables=["query", "answer", "context"],
            template="""
            Evaluate the quality of the following answer based on the query and available context.
            
            Query: {query}
            
            Context:
            {context}
            
            Answer: {answer}
            
            Rate the answer on a scale of 0-10 considering:
            - Accuracy and factual correctness
            - Completeness and comprehensiveness
            - Relevance to the query
            - Proper use of available context
            - Clarity and helpfulness
            
            Provide only a single number between 0 and 10 as your rating.
            
            Rating:
            """
        )

    async def initialize(self, force_rebuild: bool = False):
        """Initialize the RAG System with option to force rebuild"""
        if self.is_initialized and not force_rebuild:
            return
        
        logger.info("Initializing the RAG system ...")

        # Try to load existing vector store first (unless force rebuild)
        if not force_rebuild and os.path.exists(self.vector_store_path):
            try:
                await self.load_vector_store(self.vector_store_path)
                logger.info("Loaded existing vector store successfully")
                return
            except Exception as e:
                logger.warning(f"Failed to load existing vector store: {e}. Building new one...")

        # Load the documents
        documents = await self.document_loader.load_documents(self.kb_folder)

        if not documents:
            logger.warning("No documents were found in the knowledge base ...")
            self.vector_store = None
            self.is_initialized = True
            return

        # Split the documents into chunks
        split_docs = self.text_splitter.split_documents(documents)
        logger.info(f"Split {len(documents)} documents into {len(split_docs)} chunks")

        # Create the vector store with progress tracking
        logger.info("Creating embeddings (this may take a while due to rate limiting)...")
        
        try:
            self.vector_store = await self._create_vector_store_with_progress(split_docs)
        except Exception as e:
            logger.error(f"Failed to create vector store: {e}")
            raise

        # ALWAYS SAVE THE VECTOR STORE AFTER CREATION
        if self.auto_save_vector_store:
            await self.save_vector_store(self.vector_store_path)
            logger.info(f"Vector store automatically saved to {self.vector_store_path}")

        self.is_initialized = True
        logger.info("Self-RAG system initialized successfully")

    async def _create_vector_store_with_progress(self, documents: List[Document]) -> FAISS:
        
        logger.info(f"Creating embeddings for {len(documents)} documents...")
        texts = [doc.page_content for doc in documents]
        embeddings_list = await self.embeddings.aembed_documents(texts)
        
        # Convert embeddings to numpy array
        embedding_matrix = np.array(embeddings_list).astype('float32')
        
        # Create FAISS index
        dimension = embedding_matrix.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embedding_matrix)
        
        # Create docstore and index mapping
        docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
        index_to_docstore_id = {i: str(i) for i in range(len(documents))}
        
        # Create FAISS vector store
        vector_store = FAISS(
            embedding_function=self.embeddings.base_embeddings,
            index=index,
            docstore=docstore,
            index_to_docstore_id=index_to_docstore_id
        )
        
        logger.info("Vector store created successfully")
        return vector_store

    async def _should_retrieve(self, query: str) -> Tuple[bool, str]:
        """ Determine if the retrieval is needed and get reasoning"""
        chain = self.retrieval_decision_prompt | self.llm.with_config(callbacks=[self.rate_limit_callback])

        result = await chain.ainvoke({"query": query})

        # Parse the result
        lines = result.content.strip().split('\n')
        reasoning = ""
        decision = False

        for line in lines:
            if line.startswith("Reasoning:"):
                reasoning = line.replace("Reasoning:", "").strip()
            elif line.startswith("Decision:"):
                decision_text = line.replace("Decision:", "").strip()
                decision = "RETRIEVE" in decision_text.upper()
        
        return decision, reasoning

    async def _retrieve_documents(self, query: str, k:int = 5) -> List[Document]:
        """Retrieve relevant documents"""
        if not self.vector_store:
            return []

        # Use similarity search with scores
        doc_with_scores = await self.vector_store.asimilarity_search_with_score(query, k = k)
        # Filter by relevance score
        relevant_docs = [doc for doc, score in doc_with_scores if score > 0.8]

        return relevant_docs

    async def _generate_answer_with_retrieval(self, query: str, documents: List[Document]) -> str:
        """Generate answers using retrieved documents"""
        context = "\n\n".join([
            f"Document: {doc.metadata.get('file_name', 'Unknown')}\n{doc.page_content}"
            for doc in documents
        ])
        
        chain = self.rag_prompt | self.llm | StrOutputParser()
        
        result = await chain.ainvoke({
            "context": context,
            "question": query
        })
        
        return result
        
    async def _generate_answer_without_retrieval(self, query: str) -> str:
        """Generate answer without retrieval"""
        chain = self.no_retrieval_prompt | self.llm | StrOutputParser()
        
        result = await chain.ainvoke({"question": query})
        return result

    async def _reflect_on_answer(self, query: str, answer: str, documents: List[Document]) -> float:
        """Reflect on answer quality"""
        context = "\n\n".join([doc.page_content for doc in documents]) if documents else "No context provided"
        
        chain = self.reflection_prompt | self.llm.with_config(callbacks=[self.rate_limit_callback])
        
        result = await chain.ainvoke({
            "query": query,
            "answer": answer,
            "context": context
        })
        
        try:
            content = result.content if hasattr(result, 'content') else str(result)
            score_str = content.strip().split('\n')[-1]
            
            # Extract digits and decimal point from the score string
            score_chars = ''.join(c for c in score_str if c.isdigit() or c == '.')
            
            if score_chars:
                score = float(score_chars)
                return min(10.0, max(0.0, score))  # Clamp between 0 and 10
            else:
                logger.warning(f"No numeric score found in: {score_str}")
                return 5.0
                
        except (ValueError, IndexError) as e:
            logger.warning(f"Could not parse reflection score: {result}. Error: {e}")
            return 5.0

    async def query(self, query: str, force_retrieval: bool = False) -> SelfRAGResponse:
        """Process query using the SelfRAG"""
        if not self.is_initialized:
            await self.initialize()

        # Decide whether we need retrieval
        if force_retrieval:
            needs_retrieval = True
            reasoning = "Forced retrieval requested"
        else:
            needs_retrieval, reasoning = await self._should_retrieve(query)

        retrieved_docs = []
        citations = []

        # Retrieve documents if needed
        if needs_retrieval and self.vector_store:
            retrieved_docs = await self._retrieve_documents(query)
            citations = [
                doc.metadata.get('file_path', f"Document {i}")
                for i, doc in enumerate(retrieved_docs)
            ]

        # Generate answer
        if retrieved_docs:
            answer = await self._generate_answer_with_retrieval(query, retrieved_docs)
        else:
            answer = await self._generate_answer_without_retrieval(query)

        # Reflect on answer quality
        reflection_score = await self._reflect_on_answer(query, answer, retrieved_docs)

        return SelfRAGResponse(
            answer=answer,
            retrieved_docs=retrieved_docs,
            reflection_score=reflection_score,
            needs_retrieval=needs_retrieval,
            citations=list(set(citations)),  # Remove duplicates
            retrieval_decision_reasoning=reasoning
        )

    async def save_vector_store(self, path: str = None):
        """Save vector store to disk"""
        if not self.vector_store:
            logger.warning("No vector store to save")
            return
            
        save_path = path or self.vector_store_path
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        try:
            self.vector_store.save_local(save_path)
            logger.info(f"Vector store saved to {save_path}")
        except Exception as e:
            logger.error(f"Failed to save vector store: {e}")
            raise
    
    async def load_vector_store(self, path: str = None):
        from langchain_community.vectorstores import FAISS
        load_path = path or self.vector_store_path
        try:
            self.vector_store = FAISS.load_local(
                load_path,
                self.embeddings.base_embeddings,
                allow_dangerous_deserialization=True
            )
            if not isinstance(self.vector_store, FAISS):
                raise ValueError("Loaded vector store is not a valid FAISS object")
            self.is_initialized = True
            logger.info(f"Vector store loaded from {load_path}")
        except Exception as e:
            logger.error(f"Failed to load vector store from {load_path}: {e}")
            self.vector_store = None
            raise

    async def rebuild_vector_store(self):
        logger.info("Force rebuilding vector store...")
        self.vector_store = None
        await self.initialize(force_rebuild=True)

In [6]:
class BatchProcessor:
    """Process multiple queries in batch with concurrency control"""
    
    def __init__(self, rag_system: RAGSystem, max_concurrent: int = 3):
        self.rag_system = rag_system
        self.semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_query(self, query: str) -> Tuple[str, SelfRAGResponse]:
        """Process a single query with semaphore"""
        async with self.semaphore:
            response = await self.rag_system.query(query)
            return query, response
    
    async def process_batch(self, queries: List[str]) -> Dict[str, SelfRAGResponse]:
        """Process multiple queries concurrently"""
        tasks = [self.process_query(query) for query in queries]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        output = {}
        for result in results:
            if isinstance(result, Exception):
                logger.error(f"Error processing query: {result}")
            else:
                query, response = result
                output[query] = response
        
        return output


In [7]:
async def main():
    rag_system = RAGSystem(
    cohere_api_key=os.getenv("COHERE_API_KEY"),
    openrouter_api_key=os.getenv("OPENROUTER_API_KEY"),
    kb_folder="books"
    )
    
    # Initialize the system
    await rag_system.initialize()
    
    
    queries = [
    "Who is Chris Olande?",
    "What has Chris Olande studied at Kenyatta University?",
    
    # Romeo and Juliet
    "What is the central conflict in Romeo and Juliet?",
    "How does Shakespeare portray love and fate in Romeo and Juliet?",

    # Declaration of Independence
    "What are the main ideas expressed in the Declaration of Independence?",
    "How does the Declaration justify the colonies break from Britain?",

    # Frankenstein
    "How does Frankenstein's pursuit of knowledge ultimately lead to his downfall?",
    "In what ways does the creature demonstrate human qualities, and how is he rejected by the society?",

    # Pride and Prejudice
    "In what ways does Elizabeth Benner challenge the gender norms of her society?",
    "How does Mr. Darcy's character evolve, and what causes this change?",

    # Scarlett Letter
    "How does the forest contrast with the Puritan settlement, and wha does this signify?",
    "In what ways does Dimmesdale suffer due to his hidden sin?, Explain what his hidden sin was in the first place",

    # U.S Bill of rights
    "What is the difference between the sixth and seventh ammendments?",
    "How does the eighth ammendment protections relate to modern debates on criminal justice?",
    "Which rights are guaranteed by the first ammendment?"
]
    
    # Process queries individually
    for query in queries:
        print(f"\n{'='*80}")
        print(f"Query: {query}")
        print('='*80)
        
        try:
            response = await rag_system.query(query)
            
            print(f"Needs Retrieval: {response.needs_retrieval}")
            print(f"Reasoning: {response.retrieval_decision_reasoning}")
            print(f"Retrieved Documents: {len(response.retrieved_docs)}")
            print(f"Reflection Score: {response.reflection_score}/10")
            
            if response.citations:
                print(f"Citations: {response.citations}")
            
            print(f"\nAnswer:\n{response.answer}")
            
        except Exception as e:
            print(f"Error processing query: {e}")
    
    
    print(f"\n{'='*60}")
    print("BATCH PROCESSING EXAMPLE")
    print('='*60)
    
    batch_processor = BatchProcessor(rag_system, max_concurrent=3)
    batch_queries = queries[:4]  # Process first 4 queries in batch
    
    batch_results = await batch_processor.process_batch(batch_queries)
    
    for query, response in batch_results.items():
        print(f"\nBatch Query: {query}")
        print(f"Reflection Score: {response.reflection_score}/10")
        print(f"Answer Length: {len(response.answer)} characters")

if __name__ == "__main__":
    await main()

INFO:__main__:Initializing the RAG system ...
INFO:__main__:Loaded 7 documents from books
INFO:__main__:Split 7 documents into 1291 chunks
INFO:__main__:Creating embeddings (this may take a while due to rate limiting)...
INFO:__main__:Creating embeddings for 1291 documents...
INFO:__main__:Processing 1291 texts in 14 batches of 96
INFO:__main__:Processing batch 1/14 (96 texts)
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:__main__:Processing batch 2/14 (96 texts)
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:__main__:Processing batch 3/14 (96 texts)
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:__main__:Processing batch 4/14 (96 texts)
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:__main__:Processing batch 5/14 (96 texts)
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:__main__:Processing batch 6


Query: Who is Chris Olande?


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Needs Retrieval: True
Reasoning: The query "Who is Chris Olande?" asks for specific information about an individual, which suggests it requires external knowledge retrieval. This type of query typically seeks factual or biographical data that may not be readily available without accessing external sources such as databases, documents, or the internet. The answer would indeed benefit from consulting external documents or a knowledge base, as the information about Chris Olande is not general knowledge that everyone would be expected to know. Furthermore, without more context, it's unclear if Chris Olande is a public figure, historical person, or someone who has recently gained prominence, which means the query could require recent or specialized information to answer accurately.
Retrieved Documents: 5
Reflection Score: 9.0/10
Citations: ['books/olande.txt', 'books/frankenstein.txt']

Answer:
Chris Olande is a dynamic and intellectually curious student of Statistics and Programming at Ken

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


...

Needs Retrieval: True
Reasoning: The query is domain-specific, requires recent information, and benefits significantly from external knowledge sources to provide a comprehensive and accurate answer.
Retrieved Documents: 5
Reflection Score: 8.0/10
Citations: ['books/us_bill_of_rights.txt', 'books/declaration_of_independence_of_the_united_states.txt']

Answer:
The eighth amendment protections, as outlined in the provided context, specifically in the "us_bill_of_rights.txt" document, state that "Excessive bail shall not be required nor excessive fines imposed, nor cruel and unusual punishments inflicted" (VIII). This amendment is directly related to modern debates on criminal justice, particularly in the areas of bail reform, sentencing, and punishment.

In modern debates, the eighth amendment's protection against excessive bail is often cited in discussions about bail reform. Many argue that the current bail system can be unfair, particularly to low-income individuals who may not be able

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Needs Retrieval: True
Reasoning: The query seeks specific, domain-specific information about legal rights guaranteed by the First Amendment, which benefits from external documentation or a knowledge base for accuracy and comprehensiveness.
Retrieved Documents: 5
Reflection Score: 9.0/10
Citations: ['books/us_bill_of_rights.txt', 'books/declaration_of_independence_of_the_united_states.txt']

Answer:
According to the provided context, specifically in the documents "us_bill_of_rights.txt" and "declaration_of_independence_of_the_united_states.txt", the First Amendment guarantees several fundamental rights. As stated in the document "us_bill_of_rights.txt", the First Amendment reads:

"Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press, or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances."

Therefore, the rights guaranteed by

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 20


Batch Query: Who is Chris Olande?
Reflection Score: 9.0/10
Answer Length: 2416 characters

Batch Query: What has Chris Olande studied at Kenyatta University?
Reflection Score: 9.0/10
Answer Length: 793 characters

Batch Query: What is the central conflict in Romeo and Juliet?
Reflection Score: 9.0/10
Answer Length: 2208 characters

Batch Query: How does Shakespeare portray love and fate in Romeo and Juliet?
Reflection Score: 9.0/10
Answer Length: 2523 characters
